## Plackett Luce Model Inference For Estimating Worth Value from Rank 

Suppose we know the outcomes of a set of pairwise competitions between a certain group of individuals, and let $w_{ij}$ be the number of times individual $i$ beats individual $j$.

Then the **likelihood** of this set of outcomes under the **Bradley–Terry model** is:

$$
L(\mathbf{p}) = \prod_{i,j} \left[ \Pr(i > j) \right]^{w_{ij}}
$$

And the probability of $i$ beating $j$ is:

$$
\Pr(i > j) = \frac{p_i}{p_i + p_j}
$$

Substituting this into the likelihood gives:

$$
L(\mathbf{p}) = \prod_{i,j} \left( \frac{p_i}{p_i + p_j} \right)^{w_{ij}}
$$

Taking the natural logarithm of the likelihood (to get the **log-likelihood**):

$$
\ell(\mathbf{p}) = \ln L(\mathbf{p}) = \sum_{i=1}^{n} \sum_{j=1}^{n} w_{ij} \ln \left( \frac{p_i}{p_i + p_j} \right)
$$

Which simplifies to:

$$
\ell(\mathbf{p}) = \sum_{i=1}^{n} \sum_{j=1}^{n} \left[ w_{ij} \ln(p_i) - w_{ij} \ln(p_i + p_j) \right]  \quad \quad  (1)
$$

Differentiating and setting to zero, we obtained p:

$$
p_{i}' = \frac{\sum_{j} w_{ij} \frac{p_j}{p_i + p_j}}{\sum_{j} \frac{w_{ji}}{p_i + p_j}}  \quad  \quad (2)
$$


This is the log-likelihood we can maximize to estimate the worth parameters $\mathbf{p} = [p_1, p_2, \dots, p_n]$.

### Goal

To obtain the latent value (worth) of items/models from their Rank information



### Example

#### Original Table: Rank performance of model based on Metrics

| Model | Metric1 | Metric2 | Metric3 |
|:-------|:--------|:--------|:--------|
| A     | 1      | 3      | 4      |
| B     | 2      | 2      | 1      |
| C     | 3      | 1      | 5      |
| D     | 4      | 4      | 2      |
| E     | 5      | 5      | 3      |

#### Transformed Table: Competition. A was better than B once and B was better than A twice, and  so on.

|   | **A** | **B** | **C** | **D** | **E** |
|---|-------|-------|-------|-------|-------|
| **A** | 0     | 1     | 2     | 2     | 2     |
| **B** | 2     | 0     | 2     | 3     | 3     |
| **C** | 1     | 1     | 0     | 2     | 2     |
| **D** | 1     | 0     | 1     | 0     | 3     |
| **E** | 1     | 0     | 1     | 0     | 0     |


In [1]:
#creating a dataframe for the competition  (zero on the diagonal, since the model can't compete with itself)
import pandas as pd
data = {
    'A': [0, 2, 1, 1, 1],
    'B': [1, 0, 1, 0, 0],
    'C': [2, 2, 0, 1, 1],
    'D': [2, 3, 2, 0, 0],
    'E': [2, 3, 2, 3, 0]
}

# Create the DataFrame
df = pd.DataFrame(data, index=['A', 'B', 'C', 'D', 'E'])

df


,A,B,C,D,E
A,0,1,2,2,2
B,2,0,2,3,3
C,1,1,0,2,2
D,1,0,1,0,3
E,1,0,1,0,0


### Using the Closed form formula (equation 2): MLE and Iteration

In [2]:
import numpy as np

def PL_MLE(df, p=None, max_iter=10):
    """
    A function that takes a dataframe of competition between items as input and
    returns their estimated worth (numeric) using immediate Plackett-Luce Model.
    """
    import numpy as np

    # Initialize worth values to 1 if not provided
    if p is None:
        p = np.ones(len(df)) 
    labels = df.columns
    
    for iteration in range(max_iter):
        # Perform updates
        for i in range(len(p)):
            numerator = 0.0
            denominator = 0.0

            for j in range(len(p)):
                if i != j:
                    w_ij = df.iloc[i, j]
                    w_ji = df.iloc[j, i]
                    numerator += w_ij * p[j] / (p[i] + p[j])
                    denominator += w_ji / (p[i] + p[j])

            # Avoid zero division
            p[i] = numerator / denominator if denominator != 0 else 0.0

        # Normalize p to keep numerical stability
        norm = np.prod(p) ** (1 / len(p))
        p = p / norm

        result_dict = {label: float(value) for label, value in zip(labels, p)}
        print(f"Iteration {iteration + 1}: {result_dict}")

    return result_dict

In [3]:
PL_MLE(df)

Iteration 1: {'A': 0.9789135975817568, 'B': 3.9410807175369427, 'C': 1.1243671458855395, 'D': 0.9198843882773917, 'E': 0.25061029850234806}
Iteration 2: {'A': 1.3436714263825265, 'B': 4.163513214084, 'C': 1.032101669486377, 'D': 0.700573624709735, 'E': 0.24721265983589236}
Iteration 3: {'A': 1.3793839168977824, 'B': 4.012184771029157, 'C': 0.9919037613920447, 'D': 0.7309492578312271, 'E': 0.24921670530380868}
Iteration 4: {'A': 1.3733031458225664, 'B': 4.008465828428561, 'C': 1.0008872024886897, 'D': 0.7280364539715035, 'E': 0.24929704886800652}
Iteration 5: {'A': 1.3728967015460145, 'B': 4.012235336544417, 'C': 0.9999237024435776, 'D': 0.728360793248826, 'E': 0.2492655821322054}
Iteration 6: {'A': 1.37304210626391, 'B': 4.011695036352832, 'C': 1.0000092536369305, 'D': 0.7283107242804916, 'E': 0.24926856266989697}
Iteration 7: {'A': 1.3730262177830639, 'B': 4.011743594684899, 'C': 0.9999984268701013, 'D': 0.7283186409030359, 'E': 0.2492684192471034}
Iteration 8: {'A': 1.373027371049482

{'A': 1.3730272718741572,
 'B': 4.011739472790178,
 'C': 1.0000000043731134,
 'D': 0.7283176523264094,
 'E': 0.24926842911358402}

### Using Gradient Ascent with Pytorch (From Equation 1)

In [4]:
import torch
def PL_Torch(df, lr=0.01, max_iter=100):
    """
    A function that takes a dataframe of competition between items as input and
    returns their estimated worth (numeric) using immediate Plackett-Luce Model.
    """
    n = len(df)
    # Initialize p values as positive parameters (requires_grad=True)
    p = torch.ones(n, requires_grad=True, dtype=torch.float64)

    # Optimizer — gradient ascent (minimize negative log-likelihood)
    optimizer = torch.optim.Adam([p], lr=lr)

    for iteration in range(max_iter):
        optimizer.zero_grad()

        # Compute log-likelihood
        log_likelihood = 0.0
        for i in range(n):
            for j in range(n):
                if i != j:
                    w_ij = df.iloc[i, j]
                    log_likelihood += w_ij * (torch.log(p[i]) - torch.log(p[i] + p[j]))

        # Negative log-likelihood for minimization
        loss = -log_likelihood
        loss.backward()
        optimizer.step()

        # Normalize p after each step for numerical stability
        with torch.no_grad():
            p /= torch.prod(p).pow(1.0 / n)

        if (iteration + 1) % 10 == 0 or iteration == 0:
            print(f"Iteration {iteration+1}: p = {p.detach().numpy()}")

    # Return final worth estimates as a dict
    result_dict = {label: float(value) for label, value in zip(df.columns, p.detach().numpy())}
    return result_dict


In [5]:
#PL_Torch(df)  #different from the obtained answer but with some changes in lr, we obtained the same answer
PL_Torch(df, lr = 0.07)

Iteration 1: p = [1.07210442 1.07210442 1.00196675 0.93182908 0.93182908]
Iteration 10: p = [1.66856514 1.94759058 0.99111579 0.753838   0.41186676]
Iteration 20: p = [1.94051966 3.29533106 0.99305176 0.80485716 0.19565557]
Iteration 30: p = [1.31475204 3.39759888 1.02985761 0.70754567 0.30722205]
Iteration 40: p = [1.3137588  3.91685327 1.00937849 0.82072326 0.23458286]
Iteration 50: p = [1.44573008 4.21134375 0.97895317 0.66336754 0.2529158 ]
Iteration 60: p = [1.32158181 4.04060069 1.00658226 0.76083241 0.24452414]
Iteration 70: p = [1.37707522 4.01537989 1.00275751 0.71283668 0.25300537]
Iteration 80: p = [1.3847457  3.99675214 1.00165981 0.73240252 0.24629335]
Iteration 90: p = [1.36644506 3.98157507 0.9996319  0.73054349 0.25169045]
Iteration 100: p = [1.38042063 4.01855352 0.9985889  0.72604468 0.2486387 ]


{'A': 1.3804206281045805,
 'B': 4.018553515504488,
 'C': 0.998588895177912,
 'D': 0.7260446816350722,
 'E': 0.24863869780215966}

### Reference 
For more on efficient methods for computing rankings from pairwise comparisons:

**Efficient Computation of Rankings from Pairwise Comparisons**  
*M. E. J. Newman; Journal of Machine Learning Research, 2023*  
[See paper](https://jmlr.org/papers/volume24/22-1086/22-1086.pdf)


**For Quick Overview: Wikipedia: Bradley–Terry_model**  
[See wikipedia](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model)
